In [ ]:
import numpy as np
from scipy.linalg import inv, det
from scipy.integrate import odeint
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from math import ceil, floor
from scipy.special import jv
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

## Discretizing the domain

In [ ]:
R = 0.15 #m radius of the wafer
d = 0.00076 #m thickness of the wafer

Nr = 20 # amount of gridpoint in the r direction
Ntheta = 40 # amount of gridpoint in theta direction, should be a multiple of 4

N = Nr*Ntheta # total amount of gridpoint over the wafer

dtheta = 2*np.pi/(Ntheta) # angular distance between the gridpoints
dr = R/(Nr-1) # radial distance between the gridpoints

ind = [(i, j) for i in range(Nr) for j in range(Ntheta)] # list of the index of every element in the temperature vector
r = [i*dr for i in range(Nr) for j in range(Ntheta)] # list of the r-component
theta = [j*dtheta for i in range(Nr) for j in range(Ntheta)] # list of the theta-component
x = [np.abs(r[n]*np.cos(theta[n])+np.sqrt(R**2-r[n]**2*np.sin(theta[n])**2)) for n in range(N)] # list of the displacement \n
# from the left boundary, absolute value is to suppress noise in cosine
y = [r[n]*np.sin(theta[n]) for n in range(N)]

tsim = 20 # s total simulation time
dt = 0.01 # s timestep
tsteps = round(tsim/dt) # amount of timesteps

tarray = np.linspace(0, tsim, tsteps) # numpy array of all the simulated timepoints

## Material and flow properties

In [ ]:
points = np.array([[1, 0],[5, 0], [10, 0], [20, 0], [30, 0], [40, 0], [1, 25],[5, 25], [10, 25], [20, 25], [30, 25], [40, 25]])

rhovalues = np.array([1.2758, 6.3940, 12.8230, 25.7770, 38.84, 51.991, 1.1685, 5.85, 11.717, 23.492, 35.308, 51.991])
cpvalues = np.array([1005.9, 1013.9, 1023.9, 1044.3, 1065.1, 1086.2, 1006.5, 1012.9, 1021, 1037.2, 1053.6, 1070])
lvalues = np.array([0.02436, 0.024505, 0.024696, 0.025118, 0.025589, 0.026102, 0.026247, 0.026376, 0.026549, 0.026925, 0.02734,\
                    0.027789])
nuvalues = np.array([1.3496*10**-5, 2.7026*10**-6, 1.354*10**-6, 6.8059*10**-7, 4.5687*10**-7, 3.4556*10**-7, 1.5787*10**-5, \
                     3.1635*10**-6, 1.586*10**-6, 7.9812*10**-7, 5.3616*10**-7, 4.0568*10**-7])

In [ ]:
# gas properties
Tgas = 22 # Celsius temperature of the bulk gas
pgas = 1 # bar pressure of the bulk gas

nu = griddata(points, nuvalues, [pgas, Tgas], method='cubic') # m^2/s kinematic viscosity of the gas (air)
lgas = griddata(points, lvalues, [pgas, Tgas], method='cubic') # W/(m K) thermal conductivity of gas (air)
cpgas = griddata(points, cpvalues, [pgas, Tgas], method='cubic') # J/(kg K) specific heat of gas (air)
rhogas = griddata(points, rhovalues, [pgas, Tgas], method='cubic') # kg/m^3 density of gas (air)


# wafer material properties
lsilicon = 130 # W/(m K) thermal conductivity of silicon (from VDL powerpoint)
cpsilicon = 710 # J/(kg K) specific heat of silicon
rhosilicon = 2330 # kg/m^3 density of silicon

# Flow properties
U = 117# m/s speed of the flow in the uniform regime

Re = [U*x[n]/nu for n in range(N)] # list of the local reynolds number
Pr = cpgas*rhogas*nu/lgas # Prandtl number
#Nu = [0.332*Pr**(1/3)*Re[n]**(1/2) for n in range(N)] # local nusselt number
Nu = [0.0296*Pr**(1/3)*Re[n]**(4/5) for n in range(N)] # local nusselt number

h = [lgas*Nu[n]/x[n] for n in range(N)] #W/(m^2 K) local heat transfer coefficient
harray = np.nan_to_num(np.array(h)).flatten() # numpy array of the local heat transfer coefficient. Nans are made zero

## Define the evolution matrix

In [ ]:
# construct the matrix equation M T^{n+1} = Q T^{n}
alfa = lsilicon/(cpsilicon * rhosilicon) # m^2/s thermal diffusivity of silicon
beta = 1/(cpsilicon*rhosilicon*d) # just a parameter to make life easier

# initial definition of two matrices in equation: M.T(t+dt) = Q.T(t), where . mean a matrix multiplication
M = np.zeros((N, N))
Q = np.identity(N)

# definition of the elements of M
for n in range(N):
    # Row corresponding to the gridpoints where r = R. No flux boundary, T(r, th) - T(r-dr, th) = 0
    if ind[n][0] == Nr-1:
        M[n, n] = 1
        M[n, n-Ntheta] = -1
        Q[n, n] = 0    
    
    # Row corresponding to the gridpoints r = 0.
    elif ind[n][0] == 0:
        # Row corresponding to theta != 0. These are equal to their neighbour, T(0, theta) - T(0, theta-dtheta) = 0
        if ind[n][1] != 0:
            M[n, n] = 1
            M[n, n-1] = -1
            Q[n, n] = 0
        # Row corresponding to r = 0. Here the heat equation in cartesian coordinates is used with the gridpoints at
        # r = dr and theta = 0, pi/2, pi, 3pi/4
        else:            
            M[n, n] = 1+4*alfa*dt/dr**2+h[n]*beta*dt
            M[n, n+Ntheta] = -alfa*dt/dr**2
            M[n, n+int(Ntheta+(Ntheta)/4)] = -alfa*dt/dr**2 
            M[n, n+int(Ntheta+(Ntheta)/2)] = -alfa*dt/dr**2
            M[n, n+int(Ntheta+3*(Ntheta)/4)] = -alfa*dt/dr**2
            
    # Row corresponding to theta = 2pi-dtheta. Periodic boundary, T(r, 2pi) = T(r, 0)
    elif ind[n][1] == Ntheta-1:
        M[n, n] = 1+2*alfa*dt*(1/dr**2+1/(r[n]*dtheta)**2)+h[n]*beta*dt
        M[n, n+Ntheta] = -alfa*dt*(1/dr**2+1/(2*r[n]*dr))
        M[n, n-Ntheta] = -alfa*dt*(1/dr**2-1/(2*r[n]*dr))
        M[n, n-Ntheta+1] = -alfa*dt/(r[n]*dtheta)**2
        M[n, n-1] = -alfa*dt/(r[n]*dtheta)**2
        
    # Row corresponding to theta = 0. Periodic boundary, T(r, -dtheta) = T(r, 2pi)
    elif ind[n][1] == 0:
        M[n, n] = 1+2*alfa*dt*(1/dr**2+1/(r[n]*dtheta)**2)+h[n]*beta*dt
        M[n, n+Ntheta] = -alfa*dt*(1/dr**2+1/(2*r[n]*dr))
        M[n, n-Ntheta] = -alfa*dt*(1/dr**2-1/(2*r[n]*dr))
        M[n, n+1] = -alfa*dt/(r[n]*dtheta)**2
        M[n, n+Ntheta-1] = -alfa*dt/(r[n]*dtheta)**2
        
    # Rows corresponding to gridpoints on the domain and not on the boundary.
    else:        
        M[n, n] = 1+2*alfa*dt*(1/dr**2+1/(r[n]*dtheta)**2)+h[n]*beta*dt
        M[n, n+Ntheta] = -alfa*dt*(1/dr**2+1/(2*r[n]*dr))
        M[n, n-Ntheta] = -alfa*dt*(1/dr**2-1/(2*r[n]*dr))
        M[n, n+1] = -alfa*dt/(r[n]*dtheta)**2
        M[n, n-1] = -alfa*dt/(r[n]*dtheta)**2
           
# Find the matrix P = M^-1.Q. It serves in the follow equation, T(t+dt) = P.(T(t)+h*Tenv*dt) 
P = np.matmul(inv(M), Q)

## Defining initial condition

In [ ]:
# defining the matrix with the temperature vector at every tstep. Every row corresponds to a temperature vector at a certain 
# timestep
T = np.zeros((tsteps, N))

# defining the initial temperature vector
for n in range(N):
    T[0, n] = 22.8 + (0.2/0.581865)*jv(1, 1.84118*r[n]/R)*np.cos(theta[n])

## Run the simulation

In [ ]:
# define the time and the index of the first temperature vector to be computed.
t = dt
i = 1

# loop over all the simulation time
while t < tsim:
    # calculate the new temperature vector
    T[i] = np.dot(P, T[i-1]+22*dt*harray*beta)
    
    # iterate the time and index
    t += dt
    i += 1

## Plotting results
### making a heatmap of the temperature over the wafer

In [ ]:
fig1 = plt.figure()

rplot = np.zeros((Nr, Ntheta+1))
rplot[:, :Ntheta] = np.array(r).reshape((Nr, Ntheta))
rplot[:, Ntheta] = rplot[:, Ntheta-1]

thetaplot = np.zeros((Nr, Ntheta+1))
thetaplot[:, :Ntheta] = np.array(theta).reshape((Nr, Ntheta))
thetaplot[: , Ntheta] = np.ones((Nr)) * np.pi * 2

Tplot = np.zeros((Nr, Ntheta+1))
Tplot[:, :Ntheta] = T[-1].reshape((Nr, Ntheta))*0
Tplot[:, Ntheta] = Tplot[:, 0]

ax1 = plt.subplot(projection='polar')

pc = ax1.pcolormesh(thetaplot, rplot, Tplot, shading='gouraud')

cbar = fig1.colorbar(pc)
cbar.ax.set_ylabel('Temperature (C)', loc='center')
#ax1.set_title("Temperature over the wafer in Celsius")
plt.axis('off')
#plt.savefig('temperature constant')
#plt.grid()

### making a graph of the average temperature and non-uniformity against time

In [ ]:
ReL = U*R/nu
NuL = 0.037 * ReL**(4/5) * Pr**(1/3)
hL = NuL*(lgas/R)

#fig2 = plt.figure()
fig2, (ax2, ax3) = plt.subplots(1, 2, figsize=(10, 4))

ax2.plot(tarray, np.mean(T, axis=1))
ax2.plot(tarray, 22+0.8*np.exp(-hL*tarray/(rhosilicon*cpsilicon*d)), '--')
ax2.legend(['Simulation', 'Lumped mass'])
ax3.plot(tarray, np.max(T, axis=1)-np.min(T, axis=1))
ax2.set_ylabel("Average temperature over the wafer (C)")
ax2.set_xlabel("Time (s)")
ax3.set_xlabel("Time (s)")
ax3.set_ylabel('non-uniformity on the wafer')

np.max(T[-1]) - np.min(T[-1])

plt.savefig("modelat 117ms 1bar")